# Dynamic Scene

to do:
- automate intersection camera location workflow with pickle and dictionary

## Setup Sim

### setup modules, import functions and module

In [60]:
%load_ext autoreload
%autoreload 2

import carla
import math
import random
import time
import queue
import numpy as np
import cv2
import matplotlib.pyplot as plt
from functions import *

client = carla.Client('localhost', 2000)
world  = client.get_world()
bp_lib = world.get_blueprint_library()
spawn_points = world.get_map().get_spawn_points()
spectator = world.get_spectator() 

#load list for camera points
cam_perspectives = {}
try:
    with open('./cam_perspectives.pickle', 'rb') as handle:
        fresh_opened_jar_of_pickles = pickle.load(handle)
except:
    print("careful: no pickle was loaded with previously saved intersections")

#return pickled lists to carla transform objects
for key, fresh_pickle in fresh_opened_jar_of_pickles.items():
    cam_perspectives[key] = unpickle_to_carla(fresh_pickle)

##wipe intermediary lists clean
fresh_opened_jar_of_pickles = {}

# Set up the simulator in synchronous mode
settings = world.get_settings()
settings.synchronous_mode = True
#time step used throughout
ticktime = 0.10 #max 0.10
settings.fixed_delta_seconds = ticktime
settings.max_substep_delta_time = ticktime/10 #max 0.01 
world.apply_settings(settings)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


30618

In [2]:
for key in cam_perspectives:
  print(key)

inter1tv
inter1pv1
inter1pv2
inter1pv3
inter1pv4
inter1pv5
inter1pv6
inter2tv
inter2pv1
inter2pv2
inter2pv3
inter2pv4


In [8]:
print(f'lastzip: {lastzipnmbr}')

lastzip: 15


In [109]:
for sensor in world.get_actors().filter('*sensor*'):

Actor(id=1915, type=sensor.camera.instance_segmentation)
Actor(id=1914, type=sensor.camera.rgb)
Actor(id=1916, type=sensor.camera.rgb)
[]


### Set Variables for Generation

In [61]:
lastzipnmbr = 69
intersection = 'inter2'
perspectives = 1

tv_key = intersection + 'tv'

## variables for dataset generation
dist_cutoff = 90
ts = 0
iteration = 0
max_timesteps = 10

### Execute in Bulk

In [112]:
for i in range(perspectives):
    pv_key = intersection + 'pv' + str(i+1)
    print(pv_key)
    iteration = 0
    ts = 0

    ## set view
    c_pv_spawn_location = cam_perspectives[pv_key]
    c_tv_spawn_location = cam_perspectives[tv_key]

    spectator.set_transform(c_tv_spawn_location) 

    # Determine Camera Parameters
    camera_bp = bp_lib.find('sensor.camera.rgb')
    camera_bp.set_attribute('image_size_x', '1920')
    camera_bp.set_attribute('image_size_y', '1080')

    camera_bp_seg = bp_lib.find('sensor.camera.instance_segmentation')
    camera_bp_seg.set_attribute('image_size_x', '1920')
    camera_bp_seg.set_attribute('image_size_y', '1080')

    ############################################################
    ## Spawn Actors
    ############################################################
    ## Vehicles
    ## note: spawn vehicles before starting cameras to give them time to exist
    for npc in world.get_actors().filter('*vehicle*'):
        npc.destroy()
        actors = []
    # Spawn Vehicles (excluding 2 wheeled)
    for i in range(150):
        vehicle_bp = random.choice(bp_lib.filter('vehicle'))
        if int(vehicle_bp.get_attribute('number_of_wheels')) != 2:
            npc = world.try_spawn_actor(vehicle_bp, random.choice(spawn_points))
        if npc:
            actors.append(npc)
            npc.set_autopilot(True)
    #let it finish spawning
    for i in range(20):
        world.tick()

    #############################################################
    ## Sensors
    # Create a queue to store and retrieve the sensor data
    # clean up old sensors first
    for sensor in world.get_actors().filter('*sensor*'):
        world.tick()       
        sensor.destroy()
    #spawn new sensors
    camera_pv = world.spawn_actor(camera_bp, c_pv_spawn_location)
    camera_pv_seg = world.spawn_actor(camera_bp_seg, c_pv_spawn_location)
    camera_tv = world.spawn_actor(camera_bp, c_tv_spawn_location)
    ##############################################################
    ## from here every world.tick() is a frame, do not desync the frame from the timestep
    ##############################################################
    #create queue
    image_queue_tv = queue.Queue()
    image_queue_pv = queue.Queue()
    image_queue_pv_seg = queue.Queue()
    camera_pv.listen(image_queue_pv.put)
    camera_pv_seg.listen(image_queue_pv_seg.put)
    camera_tv.listen(image_queue_tv.put)

    actors = []

    ## Determine Camera Matrices to calculate World to Camera coordinates
    def build_intrinsic_matrix(w, h, fov):
        focal = w / (2.0 * np.tan(fov * np.pi / 360.0))
        K = np.identity(3)
        K[0, 0] = K[1, 1] = focal
        K[0, 2] = w / 2.0
        K[1, 2] = h / 2.0
        return K
    
    def get_image_point(loc, K, w2c):
        # Calculate 2D projection of 3D coordinate

        # Format the input coordinate (loc is a carla.Position object)
        point = np.array([loc.x, loc.y, loc.z, 1])
        # transform to camera coordinates
        point_camera = np.dot(w2c, point)

        # New we must change from UE4's coordinate system to an "standard"
        # (x, y ,z) -> (y, -z, x)
        # and we remove the fourth componebonent also
        point_camera = [point_camera[1], -point_camera[2], point_camera[0]]

        # now project 3D->2D using the camera matrix
        point_img = np.dot(K, point_camera)
        # normalize
        point_img[0] /= point_img[2]
        point_img[1] /= point_img[2]

        return int(point_img[0]), int(point_img[1])
        
        
    # Get the world to camera matrix
    world_2_camera = np.array(camera_pv.get_transform().get_inverse_matrix())
    world_2_camera_bev = np.array(camera_tv.get_transform().get_inverse_matrix())

    # Get the attributes from the camera
    image_w = camera_bp.get_attribute("image_size_x").as_int()
    image_h = camera_bp.get_attribute("image_size_y").as_int()
    fov = camera_bp.get_attribute("fov").as_float()

    # Calculate the camera intrinsic matrix
    K = build_intrinsic_matrix(image_w, image_h, fov)
      

    ########################################## 
    ## make dataset
    ##########################################

    import pickle
    import os
    import glob
    from tqdm import tqdm
    %autoreload 2

    def calculate_img_bb(bb):
        verts = [get_image_point(v, K, world_2_camera) for v in bb.get_world_vertices(npc.get_transform())]
        return verts

    def get_mask(bb, image_cropped, image_seg):
        rgba, counts = np.unique(image_cropped.reshape(-1,4), axis=0, return_counts=True)
        # Filter street ((1, 51, 115, 255) inters2)
        mask = ~np.all((rgba == np.array([1, 94, 110, 255])), axis=1)

        rgba = rgba[mask]
        counts = counts[mask]
        target_value = rgba[np.argmax(counts)]
        mask = np.all(np.asarray(image_seg) == target_value, axis=-1)
        id = target_value[0]
        if (id == 1 or id == 2 or id == 3 or id == 9):
            street_colors.append(target_value)
        return mask

    edges = [[0,1], [1,3], [3,2], [2,0], [0,4], [4,5], [5,1], [5,7], [7,6], [6,4], [6,2], [7,3]] # For drawing BBOXES

    # Delete Dataset
    files = glob.glob('output/pv/*')
    for f in files:
        os.remove(f)
    files = glob.glob('output/tv/*')
    for f in files:
        os.remove(f)
    files = glob.glob('output/seg/*')
    for f in files:
        os.remove(f)

    vehicles_pv_per_ts = dict()
    vehicles_tv_per_ts = dict()

    ########################################
    ## loop to generate frames and info
    ########################################
    pbar = tqdm(total = max_timesteps)  #progress bar wrapped around
    while True:
        world.tick()
        
        image_pv = image_queue_pv.get()
        image_tv = image_queue_tv.get()
        image_pv_seg = image_queue_pv_seg.get()
        iteration += 1
        
        frame_path_pv = 'output/pv/%06d' % image_pv.frame
        frame_path_tv = 'output/tv/%06d' % image_tv.frame
        frame_path_seg = 'output/seg/%06d' % image_pv_seg.frame
        
        # Save the image
        image_pv.save_to_disk(frame_path_pv + '.png')
        image_tv.save_to_disk(frame_path_tv + '.png')
        image_pv_seg.save_to_disk(frame_path_seg + '.png', color_converter=carla.ColorConverter.Raw)
        
        
        # Reshape the raw data into an RGB array
        img_pv = np.reshape(np.copy(image_pv.raw_data), (image_pv.height, image_pv.width, 4)) 
        img_tv = np.reshape(np.copy(image_tv.raw_data), (image_tv.height, image_tv.width, 4)) 
        img_pv_seg = np.reshape(np.copy(image_pv_seg.raw_data), (image_pv_seg.height, image_pv_seg.width, 4)) 
        
        vehicles_pv = []
        vehicles_tv = []
        
        for npc in world.get_actors().filter('*vehicle*'):
            dist = npc.get_transform().location.distance(camera_pv.get_transform().location)
            bb = npc.bounding_box
            
            # Filter for the vehicles within
            if dist > dist_cutoff:
                continue
                
            # Calculate the dot product between the forward vector
            # of the vehicle and the vector between the vehicle
            # and the other vehicle. We threshold this dot product
            # to limit to drawing bounding boxes IN FRONT OF THE CAMERA
            forward_vec = camera_pv.get_transform().get_forward_vector()
            ray = npc.get_transform().location - camera_pv.get_transform().location

            if forward_vec.dot(ray) > 1:
                bb_img = calculate_img_bb(bb)
                gcp_x, gcp_y = get_image_point(npc.get_location(), K, world_2_camera)
                fvec = npc.get_transform().get_forward_vector()
                psi = npc.get_location().x + fvec.x, npc.get_location().y + fvec.y, npc.get_location().z + fvec.z
                psi_x, psi_y = get_image_point(carla.Location(psi[0], psi[1], psi[2]), K, world_2_camera)
                gcp_x_bev, gcp_y_bev = get_image_point(npc.get_location(), K, world_2_camera_bev)
                psi_x_bev, psi_y_bev = get_image_point(carla.Location(psi[0], psi[1], psi[2]), K, world_2_camera_bev)
                
                entry_pv = {'id': npc.id, 'gcp': [gcp_x, gcp_y], 'psi': [psi_x, psi_y], 'img': frame_path_pv + '.png', 'bb': bb_img}
                entry_tv = {'id': npc.id, 'gcp': [gcp_x_bev, gcp_y_bev], 'psi': [psi_x_bev, psi_y_bev], 'img': frame_path_tv + '.png', 'bb': bb_img}
                vehicles_pv.append(entry_pv)
                vehicles_tv.append(entry_tv)
                
        #print('iteration', iteration)
        pbar.update(1)
        vehicles_pv_per_ts[iteration] = vehicles_pv
        vehicles_tv_per_ts[iteration] = vehicles_tv


        if iteration >= max_timesteps:
            break

    ##ende
    pbar.close()
    print(f'finished generating {pv_key}')
    ####################################################################################




    ######################################
    ## Package
    ######################################
    import random 
    import pickle
    import matplotlib.pyplot as plt
    import numpy as np
    from PIL import Image
    import cv2
    import numpy.typing as npt
    from dataclasses import dataclass
    %autoreload 2

    @dataclass(frozen=True)
    class Instance:
        id: int  # vehicle id from carla
        ts: int  # timestamp should also be the image name in seg, pv and tv folders
        # pv
        gcp_pv: npt.NDArray[np.int_]  # [2,] Ground contact point
        psi_pv: npt.NDArray[np.int_]  # [2,] point for the direction of the vehicle
        bb_pv: npt.NDArray[np.int_]   # [8, 2] # 3d bounding box of the vehicle
        hull_pv: npt.NDArray[np.int_]  # [n, 2] # hull of the vehicle
        image_pv: str        # path to the image
        # tv
        gcp_tv: npt.NDArray[np.int_]  # [2,] Ground contact point
        psi_tv: npt.NDArray[np.int_]  # [2,] point for the direction of the vehicle
        bb_tv: npt.NDArray[np.int_]   # [8, 2] # 3d bounding box of the vehicle
        image_tv: str        # path to the image

    street_colors = []

    base_url = './'
    w = 1980
    h = 1080

    all_instances = []

    pbar = tqdm(total = len(vehicles_pv_per_ts.keys()))          #progress bar
    for i in vehicles_pv_per_ts.keys():
        pbar.update(1)
        data_pv = vehicles_pv_per_ts[i]
        data_tv = vehicles_tv_per_ts[i]
        vehicles_pv = filter_vehicles_not_in_img(data_pv, w, h)
        #vehicles_pv = append_hull(base_url, vehicles_at_ts_filtered)
        #vehicles_pv = append_min_area_rect(vehicles_at_ts_filtered)
        fname_pv = base_url + data_pv[0]['img']             # was vehicles_at_ts_filtered
        #image_pv = Image.open(fname_pv)
        
        # Top View
        vehicles_at_ts_tv = get_vehicles_from_ts(data_tv, ts)
        v_ids = get_vehicle_ids(vehicles_pv)
        vehicles_tv = filter_tv_vehicles(data_tv, v_ids)
        if len(vehicles_tv) == 0:       # skip if no valid vehicles in frame
            continue
        fname_tv = base_url+vehicles_tv[0]['img']
        assert len(vehicles_pv) == len(vehicles_tv)

        fname = base_url+data_pv[0]['img']
        fname_seg = base_url + 'output/seg/' + fname.split('/')[-1]
        image_seg = cv2.imread(fname_seg, cv2.IMREAD_UNCHANGED)
        image_seg = clean_segmask(image_seg)

        for vehicle_pv, vehicle_tv in zip(vehicles_pv, vehicles_tv):

            hull = calculate_hull(image_seg, vehicle_pv["bb"])
            if hull is None: # skip all vehicles where the hull cant be calculated
                continue
            instance = Instance(vehicle_pv['id'], 
                                i,
                                np.array(vehicle_pv['gcp']),
                                np.array(vehicle_pv['psi']),
                                np.array(vehicle_pv['bb']),
                                np.array(np.array(hull)),
                                vehicle_pv['img'] ,
                                np.array(vehicle_tv['gcp']),
                                np.array(vehicle_tv['psi']),
                                np.array(vehicle_tv['bb']),
                                vehicle_tv['img']
                                )
            all_instances.append(instance)
            
    pbar.close()

    with open('output/instances.pickle', 'wb') as handle:
        pickle.dump(all_instances, handle, protocol=pickle.HIGHEST_PROTOCOL)

    print(f"Saved {len(all_instances)} instances to pickle for {pv_key}")

    #####################
    # zip it up
    #####################
    import shutil

    lastzipnmbr += 1
    zipfilename = "set" + str(lastzipnmbr) + '_' + pv_key

    print(zipfilename)

    shutil.make_archive(zipfilename, format='zip', root_dir='output')



inter2pv1


100%|██████████| 10/10 [00:20<00:00,  2.07s/it]


finished generating inter2pv1


100%|██████████| 10/10 [00:01<00:00,  6.99it/s]


Saved 20 instances to pickle for inter2pv1
set82_inter2pv1


### Peds (currently buggy)

In [ ]:
""" walker_bp = bp_lib.filter("walker.pedestrian.*")
controller_bp = bp_lib.find("controller.ai.walker")

for npc in world.get_actors().filter('*pedestrian*'):
    npc.destroy()

for i in range(70):
    trans = carla.Transform()
    #trans.location = carla.Location(20, 20, 0)
    trans.location = world.get_random_location_from_navigation()
    trans.location.z += 1
    
    walker = random.choice(walker_bp)
    actor = world.try_spawn_actor(walker, trans)
    if actor:
        actors.append(actor)
    world.tick()
    controller = world.try_spawn_actor(controller_bp, carla.Transform(), actor)
    controller.start()    
    world.tick()
    controller.go_to_location(world.get_random_location_from_navigation()) """